## QA RAG   
https://www.youtube.com/watch?v=jSd6pdydBNU  
https://colab.research.google.com/drive/1TZo2sgf1YFzI4_U-tGppg_ylHAR3MXF_?usp=sharing#scrollTo=DTDNFXaBSO2j

In [9]:
import os

from dotenv import load_dotenv

load_dotenv()

True

In [2]:
%pip install pymupdf
%pip install rapidocr-onnxruntime

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [8]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.chat_models import ChatOllama
from langchain.prompts import ChatPromptTemplate
from pprint import pprint

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [4]:
%pip install chromadb

Note: you may need to restart the kernel to use updated packages.


In [2]:
from langchain_community.document_loaders import PyMuPDFLoader

pdf_filepath = "datasets/[24-13-포커스-1] 글로벌 EV 시장 동향 및 전망(IEA).pdf"
loader = PyMuPDFLoader(pdf_filepath, extract_images=True)
docs = loader.load()
print(docs[0].metadata)

{'source': 'datasets/[24-13-포커스-1] 글로벌 EV 시장 동향 및 전망(IEA).pdf', 'file_path': 'datasets/[24-13-포커스-1] 글로벌 EV 시장 동향 및 전망(IEA).pdf', 'page': 0, 'total_pages': 9, 'format': 'PDF 1.4', 'title': '', 'author': 'comdr', 'subject': '', 'keywords': '', 'creator': 'Hancom PDF 1.3.0.545', 'producer': 'Hancom PDF 1.3.0.545', 'creationDate': "D:20240701134418+09'00'", 'modDate': "D:20240701134418+09'00'", 'trapped': ''}


In [23]:
print(type(docs[0]))

<class 'langchain_core.documents.base.Document'>


In [ ]:
print(len(docs))

In [40]:
%pip install python-docx

Note: you may need to restart the kernel to use updated packages.


In [10]:
from docx import Document

def get_docx_metadata(file_path):
    """
    Extracts metadata (e.g., title, author, creation date) from the docx file.
    """
    doc = Document(file_path)

    # Get document properties (metadata)
    core_props = doc.core_properties

    # Extracting the metadata details safely
    metadata = {
        'source': file_path,
        'file_path': file_path,
        'title': core_props.title or '',
        'author': core_props.author or '',
        'subject': core_props.subject or '',
        'keywords': core_props.keywords or '',
        'creator': getattr(core_props, 'creator', '') or '',  # Safely access creator
        'producer': getattr(core_props, 'producer', '') or '',  # Safely access producer
        'creationDate': str(core_props.created) or '',
        'modDate': str(core_props.modified) or ''
    }

    return metadata

# Path to your .docx file
file_path = 'datasets/tax_doc_markdown.docx'

# Extract document text
doc = Document(file_path)
text = []
for para in doc.paragraphs:
    text.append(para.text)

# Combine all paragraphs into a single string
doc_text = '\n'.join(text)

# Split the document text by double newlines ("\n\n")
split_text = doc_text.split('\n\n')

# Extract metadata
metadata = get_docx_metadata(file_path)

# Create a list of dictionaries where each item contains the split text and the associated metadata
doc_segments_with_metadata = []

for segment in split_text:
    doc_segments_with_metadata.append({
        'text': segment,
        'metadata': metadata
    })

# Output the resulting list of segments with metadata
for i, segment in enumerate(doc_segments_with_metadata):
    print(f"Segment {i+1}:")
    print("Text:", segment['text'])
    print("Metadata:", segment['metadata'])
    print("-" * 80)  # Separator for clarity

Segment 1:
Text: 소득세법
[시행 2024. 1. 1.] [법률 제19933호, 2023. 12. 31., 일부개정]
기획재정부(소득세제과(사업소득, 기타소득)) 044-215-4217
기획재정부(소득세제과(근로소득)) 044-215-4216
기획재정부(재산세제과(양도소득세)) 044-215-4314
기획재정부(금융세제과(이자소득, 배당소득)) 044-215-4236
Metadata: {'source': 'datasets/tax_doc_markdown.docx', 'file_path': 'datasets/tax_doc_markdown.docx', 'title': '', 'author': '강병진', 'subject': '', 'keywords': '', 'creator': '', 'producer': '', 'creationDate': '2024-05-04 08:52:00+00:00', 'modDate': '2024-05-18 14:10:00+00:00'}
--------------------------------------------------------------------------------
Segment 2:
Text: 제1장 총칙 <개정 2009. 12. 31.>	
Metadata: {'source': 'datasets/tax_doc_markdown.docx', 'file_path': 'datasets/tax_doc_markdown.docx', 'title': '', 'author': '강병진', 'subject': '', 'keywords': '', 'creator': '', 'producer': '', 'creationDate': '2024-05-04 08:52:00+00:00', 'modDate': '2024-05-18 14:10:00+00:00'}
--------------------------------------------------------------------------------
Segment 3:
Text: 제1조(목

In [11]:
tax_docs = []
for row in doc_segments_with_metadata:
    if len(row['text']) < 40:
        continue 
    tax_docs.append(row)

for row in tax_docs:
    print(row['text'])
    print("-"*80)

소득세법
[시행 2024. 1. 1.] [법률 제19933호, 2023. 12. 31., 일부개정]
기획재정부(소득세제과(사업소득, 기타소득)) 044-215-4217
기획재정부(소득세제과(근로소득)) 044-215-4216
기획재정부(재산세제과(양도소득세)) 044-215-4314
기획재정부(금융세제과(이자소득, 배당소득)) 044-215-4236
--------------------------------------------------------------------------------
제1조(목적) 이 법은 개인의 소득에 대하여 소득의 성격과 납세자의 부담능력 등에 따라 적정하게 과세함으로써 조세부담의 형평을 도모하고 재정수입의 원활한 조달에 이바지함을 목적으로 한다.
[본조신설 2009. 12. 31.]
[종전 제1조는 제2조로 이동 <2009. 12. 31.>]
--------------------------------------------------------------------------------
제1조의2(정의) ① 이 법에서 사용하는 용어의 뜻은 다음과 같다. <개정 2010. 12. 27., 2014. 12. 23., 2018. 12. 31.>
1. “거주자”란 국내에 주소를 두거나 183일 이상의 거소(居所)를 둔 개인을 말한다.
2. “비거주자”란 거주자가 아닌 개인을 말한다.
3. “내국법인”이란 「법인세법」 제2조제1호에 따른 내국법인을 말한다.
4. “외국법인”이란 「법인세법」 제2조제3호에 따른 외국법인을 말한다.
5. “사업자”란 사업소득이 있는 거주자를 말한다.
② 제1항에 따른 주소ㆍ거소와 거주자ㆍ비거주자의 구분은 대통령령으로 정한다.
[본조신설 2009. 12. 31.]
--------------------------------------------------------------------------------
제2조(납세의무) ① 다음 각 호의 어느 하나에 해당하는 개인은 이 법에 따라 각자의 소득에 대한 소득세를 

In [38]:
print(type(tax_docs))

<class 'list'>


In [12]:
from langchain_core.documents import Document as LangchainDocument

# Loop through each row in tax_docs and convert to langchain Document
langchain_documents = []

for row in tax_docs:
    # Create a langchain Document object
    langchain_doc = LangchainDocument(
        page_content=row['text'],  # Set the document's content (text)
        metadata=row['metadata']   # Set the document's metadata
    )
    
    # Append to the list of langchain documents
    langchain_documents.append(langchain_doc)

# Print the langchain documents
for doc in langchain_documents:
    print(f"Document: {doc.page_content}")
    print(f"Metadata: {doc.metadata}")
    print("-" * 80)  # Separator for clarity


Document: 소득세법
[시행 2024. 1. 1.] [법률 제19933호, 2023. 12. 31., 일부개정]
기획재정부(소득세제과(사업소득, 기타소득)) 044-215-4217
기획재정부(소득세제과(근로소득)) 044-215-4216
기획재정부(재산세제과(양도소득세)) 044-215-4314
기획재정부(금융세제과(이자소득, 배당소득)) 044-215-4236
Metadata: {'source': 'datasets/tax_doc_markdown.docx', 'file_path': 'datasets/tax_doc_markdown.docx', 'title': '', 'author': '강병진', 'subject': '', 'keywords': '', 'creator': '', 'producer': '', 'creationDate': '2024-05-04 08:52:00+00:00', 'modDate': '2024-05-18 14:10:00+00:00'}
--------------------------------------------------------------------------------
Document: 제1조(목적) 이 법은 개인의 소득에 대하여 소득의 성격과 납세자의 부담능력 등에 따라 적정하게 과세함으로써 조세부담의 형평을 도모하고 재정수입의 원활한 조달에 이바지함을 목적으로 한다.
[본조신설 2009. 12. 31.]
[종전 제1조는 제2조로 이동 <2009. 12. 31.>]
Metadata: {'source': 'datasets/tax_doc_markdown.docx', 'file_path': 'datasets/tax_doc_markdown.docx', 'title': '', 'author': '강병진', 'subject': '', 'keywords': '', 'creator': '', 'producer': '', 'creationDate': '2024-05-04 08:52:00+00:00', 'modDate': '2024-05-18 1

In [29]:
print(type(docs[0]))

<class 'langchain_core.documents.base.Document'>


In [6]:
langchain_documents[0].metadata

{'source': 'datasets/tax_doc_markdown.docx',
 'file_path': 'datasets/tax_doc_markdown.docx',
 'title': '',
 'author': '강병진',
 'subject': '',
 'keywords': '',
 'creator': '',
 'producer': '',
 'creationDate': '2024-05-04 08:52:00+00:00',
 'modDate': '2024-05-18 14:10:00+00:00'}

In [7]:
%pip install langchain_upstage

  Using cached pypdf-4.3.1-py3-none-any.whl.metadata (7.4 kB)
Using cached pypdf-4.3.1-py3-none-any.whl (295 kB)
  Attempting uninstall: pypdf
    Found existing installation: pypdf 5.1.0
    Uninstalling pypdf-5.1.0:
      Successfully uninstalled pypdf-5.1.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llama-index-readers-file 0.4.1 requires pypdf<6.0.0,>=5.1.0, but you have pypdf 4.3.1 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [13]:
from langchain_upstage import UpstageEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
import os

UPSTAGE_API_KEY = os.environ["UPSTAGE_API_KEY"]

embedding = UpstageEmbeddings(model="solar-embedding-1-large",
                              api_key=UPSTAGE_API_KEY)

In [14]:
# tokenizer
from tokenizers import Tokenizer
tokenizer = Tokenizer.from_pretrained("upstage/solar-1-mini-tokenizer")

In [15]:
# Token 수를 기준으ㄹ 문서를 청크 단위로 분할
text_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n"],
    chunk_size=500,
    chunk_overlap=100,
)

split_docs = text_splitter.split_documents(langchain_documents)
print(len(split_docs))
print(split_docs[0])

870
page_content='소득세법
[시행 2024. 1. 1.] [법률 제19933호, 2023. 12. 31., 일부개정]
기획재정부(소득세제과(사업소득, 기타소득)) 044-215-4217
기획재정부(소득세제과(근로소득)) 044-215-4216
기획재정부(재산세제과(양도소득세)) 044-215-4314
기획재정부(금융세제과(이자소득, 배당소득)) 044-215-4236' metadata={'source': 'datasets/tax_doc_markdown.docx', 'file_path': 'datasets/tax_doc_markdown.docx', 'title': '', 'author': '강병진', 'subject': '', 'keywords': '', 'creator': '', 'producer': '', 'creationDate': '2024-05-04 08:52:00+00:00', 'modDate': '2024-05-18 14:10:00+00:00'}


In [16]:
vectorstore = Chroma.from_documents(documents=split_docs, 
                                    embedding=embedding)

retriever = vectorstore.as_retriever()

# Prompt
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

# Chat Model
llm = ChatOllama(model="llama3.1:8b")


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


# RAG Chain
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# Chain 실행
response = rag_chain.invoke("이 문서의 특징을 3가지 요점으로 설명해주세요.")
print(response)

/tmp/ipykernel_10960/1593712833.py:16: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  llm = ChatOllama(model="llama3.1:8b")


이 문서의 특징은 다음과 같습니다.

1. 신용카드매출전표의 사실 여부를 확인하는 조항이다. 
2. 사업자의 간이지급명세서 제출과 관련된 사항들을 규정한 조항이다. 
3. 현금영수증가맹점 가입에 관한 조항이다.


In [18]:
# Chain 실행
response = rag_chain.invoke("연봉 5천만원인 거주자의 종합소득세는?")
print(response)

연봉 5천만원을 종합소득으로 본다면, 1,400만원을 초과하는 금액은 4천만원입니다. 따라서, 그 금액의 15%를 더한 금액은 다음과 같습니다.

5,000만원 이하인 경우에 해당하는 세율을 적용한다고 해석합니다. 

그렇다면 연봉 5천만원의 종합소득세는 다음과 같이 계산할 수 있습니다.

1. 4천만원의 15% = 600만원
2. 그 세액은 84만원 + (1,400만원 초과하는 금액의 15%) = 84만원 + 600만원 = 684만원

따라서 연봉 5천만원인 거주자의 종합소득세는 약 684만원입니다.


## Test Data 만들기 - AutoRAG 활용

In [17]:
import nest_asyncio
nest_asyncio.apply()

In [14]:
%pip install autorag

  Using cached pypdf-5.1.0-py3-none-any.whl.metadata (7.2 kB)
INFO: pip is looking at multiple versions of langchain-upstage to determine which version is compatible with other requirements. This could take a while.
  Using cached langchain_upstage-0.3.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached langchain_upstage-0.2.2-py3-none-any.whl.metadata (3.3 kB)
  Using cached langchain_upstage-0.2.1-py3-none-any.whl.metadata (3.3 kB)
  Using cached langchain_upstage-0.2.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached langchain_upstage-0.1.8-py3-none-any.whl.metadata (3.4 kB)
  Using cached langchain_upstage-0.1.7-py3-none-any.whl.metadata (3.3 kB)
  Using cached langchain_upstage-0.1.6-py3-none-any.whl.metadata (3.3 kB)
INFO: pip is still looking at multiple versions of langchain-upstage to determine which version is compatible with other requirements. This could take a while.
  Using cached langchain_upstage-0.1.5-py3-none-any.whl.metadata (1.9 kB)
  Using cached langchain_upstage

In [15]:
%pip install --upgrade autorag langchain llama_index

  Using cached llama_index-0.12.9-py3-none-any.whl.metadata (11 kB)
  Using cached llama_index_agent_openai-0.4.1-py3-none-any.whl.metadata (726 bytes)
  Using cached llama_index_cli-0.4.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached llama_index_core-0.12.9-py3-none-any.whl.metadata (2.5 kB)
  Using cached llama_index_embeddings_openai-0.3.1-py3-none-any.whl.metadata (684 bytes)
  Using cached llama_index_llms_openai-0.3.12-py3-none-any.whl.metadata (3.3 kB)
  Using cached llama_index_multi_modal_llms_openai-0.4.2-py3-none-any.whl.metadata (726 bytes)
  Using cached llama_index_program_openai-0.3.1-py3-none-any.whl.metadata (764 bytes)
  Using cached llama_index_question_gen_openai-0.3.0-py3-none-any.whl.metadata (783 bytes)
  Using cached llama_index_readers_file-0.4.1-py3-none-any.whl.metadata (5.4 kB)
  Using cached llama_index_readers_llama_parse-0.4.0-py3-none-any.whl.metadata (3.6 kB)
INFO: pip is looking at multiple versions of llama-index-indices-managed-llama-cloud to det

In [20]:
import pandas as pd
import uuid
from langchain.schema import Document


# Document 객체들을 pandas DataFrame으로 변환하는 함수
def documents_to_dataframe(docs):
    data = []
    for doc in docs:
        # 고유한 doc_id 생성
        doc_id = str(uuid.uuid4())
        
        # doc_id, contents, metadata를 DataFrame에 추가
        data.append({
            'doc_id': doc_id,
            'contents': doc.page_content,
            'metadata': doc.metadata
        })
    return pd.DataFrame(data)

# split_docs를 pandas DataFrame으로 변환
corpus_df = documents_to_dataframe(split_docs)

# DataFrame을 Parquet 파일로 저장
corpus_df.to_parquet('./corpus_data/ev_car_market.parquet')

# 데이터프레임의 상위 5개 행 출력
corpus_df.head()

,doc_id,contents,metadata
0,56a00403-1c41-4e9e-9613-4c4efcebc6ae,"소득세법\n[시행 2024. 1. 1.] [법률 제19933호, 2023. 12. ...","{'source': 'datasets/tax_doc_markdown.docx', '..."
1,35076fd3-0e36-485c-81c1-a03957a22ea5,제1조(목적) 이 법은 개인의 소득에 대하여 소득의 성격과 납세자의 부담능력 등에 ...,"{'source': 'datasets/tax_doc_markdown.docx', '..."
2,1cdcdd6b-939e-4c5d-9976-508819342b89,제1조의2(정의) ① 이 법에서 사용하는 용어의 뜻은 다음과 같다. <개정 2010...,"{'source': 'datasets/tax_doc_markdown.docx', '..."
3,1692980d-3334-4fa5-849a-d2223536f91c,제2조(납세의무) ① 다음 각 호의 어느 하나에 해당하는 개인은 이 법에 따라 각자...,"{'source': 'datasets/tax_doc_markdown.docx', '..."
4,54ad2710-7a40-4af6-b8db-c420b8ce0cb1,"2. 비거주자\n3. 내국법인\n4. 외국법인의 국내지점 또는 국내영업소(출장소, ...","{'source': 'datasets/tax_doc_markdown.docx', '..."


In [21]:
for content in corpus_df.contents:
    print(content)
    print("=-----------------------------------------------------------------")

소득세법
[시행 2024. 1. 1.] [법률 제19933호, 2023. 12. 31., 일부개정]
기획재정부(소득세제과(사업소득, 기타소득)) 044-215-4217
기획재정부(소득세제과(근로소득)) 044-215-4216
기획재정부(재산세제과(양도소득세)) 044-215-4314
기획재정부(금융세제과(이자소득, 배당소득)) 044-215-4236
=-----------------------------------------------------------------
제1조(목적) 이 법은 개인의 소득에 대하여 소득의 성격과 납세자의 부담능력 등에 따라 적정하게 과세함으로써 조세부담의 형평을 도모하고 재정수입의 원활한 조달에 이바지함을 목적으로 한다.
[본조신설 2009. 12. 31.]
[종전 제1조는 제2조로 이동 <2009. 12. 31.>]
=-----------------------------------------------------------------
제1조의2(정의) ① 이 법에서 사용하는 용어의 뜻은 다음과 같다. <개정 2010. 12. 27., 2014. 12. 23., 2018. 12. 31.>
1. “거주자”란 국내에 주소를 두거나 183일 이상의 거소(居所)를 둔 개인을 말한다.
2. “비거주자”란 거주자가 아닌 개인을 말한다.
3. “내국법인”이란 「법인세법」 제2조제1호에 따른 내국법인을 말한다.
4. “외국법인”이란 「법인세법」 제2조제3호에 따른 외국법인을 말한다.
5. “사업자”란 사업소득이 있는 거주자를 말한다.
② 제1항에 따른 주소ㆍ거소와 거주자ㆍ비거주자의 구분은 대통령령으로 정한다.
[본조신설 2009. 12. 31.]
=-----------------------------------------------------------------
제2조(납세의무) ① 다음 각 호의 어느 하나에 해당하는 개인은 이 법에 따라 각자의 소득에 대한 소득세를 납부할 의무를 진다.
1. 거주자
2. 비거주자로서 국내원천소득(國內源泉所得

In [17]:
%pip install langchain_chroma

Note: you may need to restart the kernel to use updated packages.


In [18]:
%pip install langchain_ollama

Note: you may need to restart the kernel to use updated packages.


In [20]:
%pip install --upgrade llama_index

  Using cached llama_index-0.12.9-py3-none-any.whl.metadata (11 kB)
  Using cached llama_index_agent_openai-0.4.1-py3-none-any.whl.metadata (726 bytes)
  Using cached llama_index_cli-0.4.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached llama_index_core-0.12.9-py3-none-any.whl.metadata (2.5 kB)
  Using cached llama_index_embeddings_openai-0.3.1-py3-none-any.whl.metadata (684 bytes)
  Using cached llama_index_llms_openai-0.3.12-py3-none-any.whl.metadata (3.3 kB)
  Using cached llama_index_multi_modal_llms_openai-0.4.2-py3-none-any.whl.metadata (726 bytes)
  Using cached llama_index_program_openai-0.3.1-py3-none-any.whl.metadata (764 bytes)
  Using cached llama_index_question_gen_openai-0.3.0-py3-none-any.whl.metadata (783 bytes)
  Using cached llama_index_readers_file-0.4.1-py3-none-any.whl.metadata (5.4 kB)
  Using cached llama_index_readers_llama_parse-0.4.0-py3-none-any.whl.metadata (3.6 kB)
INFO: pip is looking at multiple versions of llama-index-indices-managed-llama-cloud to det

In [35]:
from langchain_ollama.llms import OllamaLLM

In [36]:
import pandas as pd
import uuid

# Load your corpus
corpus_df = pd.read_parquet('corpus_data/ev_car_market.parquet')

# Initialize the LLM (Ollama in this case)
llm = OllamaLLM(model='llama3.1:8b', temperature=0.2)

# Define a function to generate questions and answers
def generate_qa_pairs(text, num_questions=1):
    prompt = f"""
    Generate question and answer pairs for the following passage.

    Passage:
    {text}

    Number of questions to generate: {num_questions}

    Example:
    [Q]: 이 책의 글쓴이는 누구인가요?
    [A]: 저자의 이름은 홍길동입니다.

    Result:
    """

    response = llm.invoke(prompt)
    
    return response 

# Create an empty list to hold the results
qa_pairs = []

# Loop through each row of the corpus to generate Q&A pairs
for idx, row in corpus_df.iterrows():
    # 너무 많이 생성되어서, 30개만 만들기로 했다. 
    if len(qa_pairs) > 30:
        break

    text = row['contents']  # Assuming 'content' column holds the text
    qid = str(uuid.uuid4())  # Generate a unique ID for each question set
    
    # Generate the Q&A pairs (you can specify the number of questions per content here)
    question_answer = generate_qa_pairs(text, num_questions=1)  # Generate 1 Q&A pair for each content
    
    # Create the corresponding retrieval_ground_truth (retrieval_gt) -- mock data for now
    retrieval_gt = [[str(uuid.uuid4())]]  # Example format for retrieval ground truth
    
    # Split the response into query (question) and generation_gt (answer)

    """response 형태 :: 
    Here are the question and answer pairs for the passage:

    [Q]: 소득세법에 대한 문의는 어디로 하죠?
    [A]: 기획재정부(소득세제과) 044-215-4217, 044-215-4216, 044-215-4314, 044-215-4236 번으로 연락해 주세요.
    """
    print(question_answer)
    for sentence in question_answer.split("\n"):
        if "[Q]" in sentence:
            query = sentence 
            continue
        if "[A]" in sentence:
            generation_gt = sentence
            break
    
    # Append the result to the list
    qa_pairs.append({
        'qid': qid,
        'retrieval_gt': retrieval_gt,
        'query': query,
        'generation_gt': generation_gt
    })

# Convert the results to a DataFrame
qa_df = pd.DataFrame(qa_pairs)

# Display the first few rows of the resulting DataFrame
qa_df.head()

Here are the question and answer pairs for the passage:

[Q]: 소득세법에 대한 문의는 어디로 하세요?
[A]: 기획재정부(소득세제과) 044-215-4217, 044-215-4216, 044-215-4314, 044-215-4236로 연락해 주세요.
Here are the question and answer pairs for the passage:

[Q]: 이 법의 목적은 무엇입니까?
[A]: 개인의 소득에 대하여 적정하게 과세함으로써 조세부담의 형평을 도모하고 재정수입의 원활한 조달에 이바지합니다.
Here are the question and answer pairs for the given passage:

[Q]: 이 법에서 "거주자"란 무엇을 말하는가?
[A]: 국내에 주소를 두거나 183일 이상의 거소(居所)를 둔 개인을 말한다.

Let me know if you'd like me to generate more!
Here are the question and answer pairs for the passage:

[Q]: 이 법에 따라 소득세를 납부할 의무가 있는 사람들은 누구입니까?
[A]: 거주자, 비거주자로서 국내원천소득이 있는 개인, 거주자, 비거주자, 내국법인, 외국법인의 국내지점 또는 국내영업소 등입니다.

Note: I generated 5 questions and answers based on the passage, but you requested only 1. If you want me to generate more or fewer pairs, please let me know!
Here are the question and answer pairs for the given passage:

[Q]: 이 법에서 "비거주자"는 무엇을 의미합니까?
[A]: 비거주자는 국내에 주사무소 또는 사업의 실질적 관리장소를 둔 법인 아닌 단체를 말합니다.
Here are the question and

,qid,retrieval_gt,query,generation_gt
0,571cf702-72f7-4d95-9174-c77cd312ae4b,[[eb4737be-8d8d-4765-b2a9-e7252e863ff9]],[Q]: 소득세법에 대한 문의는 어디로 하세요?,"[A]: 기획재정부(소득세제과) 044-215-4217, 044-215-4216, ..."
1,959b8818-100f-47d4-8171-350660aeb765,[[c20a91e1-5ca9-4bb9-8b97-91c8a30ad23b]],[Q]: 이 법의 목적은 무엇입니까?,[A]: 개인의 소득에 대하여 적정하게 과세함으로써 조세부담의 형평을 도모하고 재정...
2,c83d8fc3-2887-4c11-9bbb-6b2b075f63e0,[[43c5faeb-e8e2-4c49-83aa-645f73dcd06c]],"[Q]: 이 법에서 ""거주자""란 무엇을 말하는가?",[A]: 국내에 주소를 두거나 183일 이상의 거소(居所)를 둔 개인을 말한다.
3,930818ee-b01a-4e2b-b50c-0f580fd32fe6,[[c32252ec-cbdf-4687-8562-8dca8857f5d0]],[Q]: 이 법에 따라 소득세를 납부할 의무가 있는 사람들은 누구입니까?,"[A]: 거주자, 비거주자로서 국내원천소득이 있는 개인, 거주자, 비거주자, 내국법..."
4,e3fedb4a-d534-43a3-b69d-239d044de3e4,[[0a976687-f590-4553-a468-d4c315fa2aac]],"[Q]: 이 법에서 ""비거주자""는 무엇을 의미합니까?",[A]: 비거주자는 국내에 주사무소 또는 사업의 실질적 관리장소를 둔 법인 아닌 단...


In [24]:
print(qa_df['generation_gt'].value_counts())

generation_gt
[[A]: 그 주식 또는 출자의 액면가액(무액면주식의 경우에는 해당 주식을 발행하는 법인의 자본금을 발행주식총수로 나누어 계산한 금액)을 그 주식 또는 출자의 취득에 사용한 금액으로 본다.]    2
[[A]: 같은 법 제13조제1항제1호에 따른 토지의 재평가차액에 상당하는 금액을 제외합니다.]                                                         2
[[A]: 제56조, 제56조의2, 제57조, 제57조의2, 제58조, 제59조 및 제59조의2부터 제59조의4까지의 규정에 따른 세액공제를 적용합니다.]                        2
[[A]: 바. 대통령령으로 정하는 작물재배업에서 발생하는 소득입니다.]                                                                      2
[[A]: 투자신탁과 집합투자기구, 집합투자업겸영보험회사의 특별계정입니다.]                                                                    2
                                                                                                             ..
[[A]: 「국군포로의 송환 및 대우 등에 관한 법률」에 따릅니다.]                                                                        1
[[A]: 거입니다.]                                                                                                  1
[[A]: 외국정부 또는 대통령령으로 정하는 국제기관에서 근무하는 사람들입니다.]                                             

In [2]:
import pandas as pd

In [37]:
qa_df.to_csv('corpus_data/tax_ex_qa.csv', index=False)

In [38]:
# 직접 수정한 QA 데이터를 불러옴
qa_data = pd.read_csv('corpus_data/tax_ex_qa.csv')
qa_data.head()

,qid,retrieval_gt,query,generation_gt
0,571cf702-72f7-4d95-9174-c77cd312ae4b,[['eb4737be-8d8d-4765-b2a9-e7252e863ff9']],[Q]: 소득세법에 대한 문의는 어디로 하세요?,"[A]: 기획재정부(소득세제과) 044-215-4217, 044-215-4216, ..."
1,959b8818-100f-47d4-8171-350660aeb765,[['c20a91e1-5ca9-4bb9-8b97-91c8a30ad23b']],[Q]: 이 법의 목적은 무엇입니까?,[A]: 개인의 소득에 대하여 적정하게 과세함으로써 조세부담의 형평을 도모하고 재정...
2,c83d8fc3-2887-4c11-9bbb-6b2b075f63e0,[['43c5faeb-e8e2-4c49-83aa-645f73dcd06c']],"[Q]: 이 법에서 ""거주자""란 무엇을 말하는가?",[A]: 국내에 주소를 두거나 183일 이상의 거소(居所)를 둔 개인을 말한다.
3,930818ee-b01a-4e2b-b50c-0f580fd32fe6,[['c32252ec-cbdf-4687-8562-8dca8857f5d0']],[Q]: 이 법에 따라 소득세를 납부할 의무가 있는 사람들은 누구입니까?,"[A]: 거주자, 비거주자로서 국내원천소득이 있는 개인, 거주자, 비거주자, 내국법..."
4,e3fedb4a-d534-43a3-b69d-239d044de3e4,[['0a976687-f590-4553-a468-d4c315fa2aac']],"[Q]: 이 법에서 ""비거주자""는 무엇을 의미합니까?",[A]: 비거주자는 국내에 주사무소 또는 사업의 실질적 관리장소를 둔 법인 아닌 단...


In [19]:
qa_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   qid            150 non-null    object
 1   retrieval_gt   150 non-null    object
 2   query          150 non-null    object
 3   generation_gt  150 non-null    object
dtypes: object(4)
memory usage: 4.8+ KB


In [28]:
type(qa_data['retrieval_gt'][0])

str

In [29]:
type(qa_data['generation_gt'][0])

str

In [20]:
import ast
for col in ['retrieval_gt', 'generation_gt']:
    qa_data[col] = qa_data[col].apply(ast.literal_eval)

qa_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   qid            150 non-null    object
 1   retrieval_gt   150 non-null    object
 2   query          150 non-null    object
 3   generation_gt  150 non-null    object
dtypes: object(4)
memory usage: 4.8+ KB


In [31]:
type(qa_data['retrieval_gt'][0])

list

In [32]:
type(qa_data['generation_gt'][0])

list

## RAGAS - RAG Evaluation

In [33]:
# Chain 테스트
question = "연봉 5천만원인 거주자의 종합소득세는?"
response = rag_chain.invoke("이 책의 출판사는 어디인가요?")
print(response)

질문에 제시된 문헌에는 "이 책의 출판사"와 관련된 정보가 없습니다. 다만, 관련된 법률에서 금융회사, 신탁업자, 외국법인 등의 경우와 관련된 내용은 포함되어 있습니다.


In [34]:
question = "연봉 5천만원인 거주자의 종합소득세는?"
response = rag_chain.invoke(question)
print(response)

1,400만원 초과 5,000만원 이하일 때 세율은 다음과 같습니다.

84만원 + (1,400만원을 초과하는 금액의 15퍼센트)

연봉이 5천만원인 경우에, 과세표준은 1,400만원 + 3,600만원 = 4,990만원입니다. 

따라서 종합소득산출세액은 다음과 같습니다.

84만원 + (1,400만원을 초과하는 금액의 15퍼센트) = 
84만원 + (3,600만원의 15퍼센트) = 
84만원 + (540만원) = 
624만원

결과적으로, 연봉 5천만원인 거주자의 종합소득세는 624만원입니다.


In [39]:
qa_data = qa_data[['query', 'generation_gt']]
qa_data.columns = ['question', 'ground_truth']
qa_data.head()

,question,ground_truth
0,[Q]: 소득세법에 대한 문의는 어디로 하세요?,"[A]: 기획재정부(소득세제과) 044-215-4217, 044-215-4216, ..."
1,[Q]: 이 법의 목적은 무엇입니까?,[A]: 개인의 소득에 대하여 적정하게 과세함으로써 조세부담의 형평을 도모하고 재정...
2,"[Q]: 이 법에서 ""거주자""란 무엇을 말하는가?",[A]: 국내에 주소를 두거나 183일 이상의 거소(居所)를 둔 개인을 말한다.
3,[Q]: 이 법에 따라 소득세를 납부할 의무가 있는 사람들은 누구입니까?,"[A]: 거주자, 비거주자로서 국내원천소득이 있는 개인, 거주자, 비거주자, 내국법..."
4,"[Q]: 이 법에서 ""비거주자""는 무엇을 의미합니까?",[A]: 비거주자는 국내에 주사무소 또는 사업의 실질적 관리장소를 둔 법인 아닌 단...


In [40]:
qa_data['answer'] = qa_data['question'].apply(lambda x: rag_chain.invoke(x) )
qa_data['contexts'] = qa_data['question'].apply(lambda x: [d.page_content for d in retriever.get_relevant_documents(x)] )
qa_data.head()

,question,ground_truth,answer,contexts
0,[Q]: 소득세법에 대한 문의는 어디로 하세요?,"[A]: 기획재정부(소득세제과) 044-215-4217, 044-215-4216, ...","[Answer]: 세무서장, 지방국세청장 또는 국세청장",[제171조(자문) 세무서장ㆍ지방국세청장 또는 국세청장은 소득세에 관한 신고ㆍ결정ㆍ...
1,[Q]: 이 법의 목적은 무엇입니까?,[A]: 개인의 소득에 대하여 적정하게 과세함으로써 조세부담의 형평을 도모하고 재정...,"Unfortunately, the provided context does not e...","[부칙 <제19933호,2023. 12. 31.>\n제1조(시행일) 이 법은 202..."
2,"[Q]: 이 법에서 ""거주자""란 무엇을 말하는가?",[A]: 국내에 주소를 두거나 183일 이상의 거소(居所)를 둔 개인을 말한다.,[A]: 국내에 주소를 두거나 183일 이상의 거소(居所)를 둔 개인을 말한다.,[제1조의2(정의) ① 이 법에서 사용하는 용어의 뜻은 다음과 같다. <개정 201...
3,[Q]: 이 법에 따라 소득세를 납부할 의무가 있는 사람들은 누구입니까?,"[A]: 거주자, 비거주자로서 국내원천소득이 있는 개인, 거주자, 비거주자, 내국법...","제2조의①에서 말하는대로, 납세의무자는 다음과 같습니다.\n\n1. 거주자\n2. ...",[제2조(납세의무) ① 다음 각 호의 어느 하나에 해당하는 개인은 이 법에 따라 각...
4,"[Q]: 이 법에서 ""비거주자""는 무엇을 의미합니까?",[A]: 비거주자는 국내에 주사무소 또는 사업의 실질적 관리장소를 둔 법인 아닌 단...,[A]: 거주자가 아닌 개인을 말합니다.,[제1조의2(정의) ① 이 법에서 사용하는 용어의 뜻은 다음과 같다. <개정 201...


In [47]:
from datasets import Dataset 

dataset = Dataset.from_pandas(qa_data.head(10))

dataset[0]

{'question': '[Q]: 소득세법에 대한 문의는 어디로 하세요?',
 'ground_truth': '[A]: 기획재정부(소득세제과) 044-215-4217, 044-215-4216, 044-215-4314, 044-215-4236로 연락해 주세요.',
 'answer': '[Answer]: 세무서장, 지방국세청장 또는 국세청장',
 'contexts': ['제171조(자문) 세무서장ㆍ지방국세청장 또는 국세청장은 소득세에 관한 신고ㆍ결정ㆍ경정 또는 조사를 할 때 필요하면 사업자로 조직된 동업조합과 이에 준하는 단체 또는 해당 사업에 관한 사정에 정통(精通)한 자에게 소득세에 관한 사항을 자문할 수 있다.\n[전문개정 2009. 12. 31.]',
  '제75조(세액감면 신청) ① 제59조의5제1항에 따라 소득세를 감면받으려는 거주자는 제69조, 제70조, 제70조의2 또는 제74조에 따른 신고와 함께 대통령령으로 정하는 바에 따라 납세지 관할 세무서장에게 신청하여야 한다. <개정 2013. 1. 1., 2014. 1. 1.>\n② 제59조의5제1항제1호에 따라 근로소득에 대한 감면을 받으려는 자는 대통령령으로 정하는 바에 따라 관할 세무서장에게 신청하여야 한다.<개정 2014. 1. 1.>\n[전문개정 2009. 12. 31.]',
  '제11조(과세 관할) 소득세는 제6조부터 제10조까지의 규정에 따른 납세지를 관할하는 세무서장 또는 지방국세청장이 과세한다.\n[전문개정 2009. 12. 31.]',
  '제164조(지급명세서의 제출) ① 제2조에 따라 소득세 납세의무가 있는 개인에게 다음 각 호의 어느 하나에 해당하는 소득을 국내에서 지급하는 자(법인, 제127조제5항 또는 제7항에 따라 소득의 지급을 대리하거나 그 지급 권한을 위임 또는 위탁받은 자 및 제150조에 따른 납세조합, 제7조 또는 「법인세법」 제9조에 따라 원천징수세액의 납세지를 본점 또는 주사무소의 소재지로 하는 자와 「부가가치세법」 제8조제3항 후단에 따른 사업자 단위 과세 사업자

In [48]:
print(type(dataset[0]['ground_truth']))

<class 'str'>


In [40]:
%pip install ragas

Note: you may need to restart the kernel to use updated packages.


In [49]:
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
)

In [51]:
from ragas import evaluate
from langchain_community.chat_models import ChatOllama
from langchain_community.embeddings.ollama import OllamaEmbeddings

langchain_llm =  ChatOllama(model="llama3.1:8b")
langchain_embeddings = OllamaEmbeddings(model="llama3.1:8b")

result = evaluate(
    dataset,
    metrics = [
        faithfulness,
        answer_relevancy,
        context_recall,
        context_precision,
    ],
    llm=langchain_llm, 
    embeddings=langchain_embeddings,
    raise_exceptions=False,
)

result

Evaluating:   0%|          | 0/40 [00:00<?, ?it/s]

Exception raised in Job[0]: TimeoutError()
Exception raised in Job[1]: TimeoutError()
Exception raised in Job[3]: TimeoutError()
Exception raised in Job[4]: TimeoutError()
Exception raised in Job[5]: TimeoutError()
Exception raised in Job[7]: TimeoutError()
Exception raised in Job[8]: TimeoutError()
Exception raised in Job[9]: TimeoutError()
Exception raised in Job[11]: TimeoutError()
Exception raised in Job[12]: TimeoutError()
Exception raised in Job[13]: TimeoutError()
Exception raised in Job[14]: TimeoutError()
Exception raised in Job[15]: TimeoutError()
Exception raised in Job[16]: TimeoutError()
Exception raised in Job[17]: TimeoutError()
Exception raised in Job[18]: TimeoutError()
Exception raised in Job[19]: TimeoutError()
Exception raised in Job[20]: TimeoutError()
Exception raised in Job[21]: TimeoutError()
Exception raised in Job[23]: TimeoutError()
Exception raised in Job[24]: TimeoutError()
Exception raised in Job[25]: TimeoutError()
Exception raised in Job[26]: TimeoutErro

{'faithfulness': nan, 'answer_relevancy': nan, 'context_recall': 0.6000, 'context_precision': nan}

### Ollama3 모델을 활용한 RAG Chain 

In [ ]:
# Prompt
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

# Chat Model
llm = ChatOllama(model="llama3.1:8b")

rag_chain2 = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

qa_data2 = qa_data.copy()
qa_data2['answer'] = qa_data2['question'].apply(lambda x: rag_chain2.invoke(x) )
qa_data2.head()

In [ ]:
dataset2 = Dataset.from_pandas(qa_data2)

result2 = evaluate(
    dataset2,
    metrics = [
        faithfulness,
        answer_relevancy,
        context_recall,
        context_precision,
    ],
    llm=langchain_llm, 
    embeddings=langchain_embeddings,
    raise_exceptions=False,
)

result2

### 모델 성능 비교 

In [ ]:
df1 = pd.DataFrame(list(result.items()), columns=['Metric', 'Qwen2'])
df2 = pd.DataFrame(list(result2.items()), columns=['Metric', 'Gemma2'])

df_result = pd.merge(df1, df2, on='Metric')
df_result